[Reference](https://medium.com/nerd-for-tech/introducing-diagrams-55b16fa805b9)

In [1]:
pip install diagrams

     |████████████████████████████████| 16.1MB 322kB/s 
     |████████████████████████████████| 102kB 7.5MB/s 
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7667 sha256=04415827fcbab10f87f6d2f386992dfa315cee904487755a4a752b756abf33c6
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built contextvars
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [2]:
from diagrams import Cluster, Diagram
from diagrams.aws.compute import ECS
from diagrams.aws.database import ElastiCache, RDS
from diagrams.aws.network import ELB
from diagrams.aws.network import Route53

with Diagram("Clustered Web Services", show=False):
    dns = Route53("dns")
    lb = ELB("lb")

    with Cluster("Services"):
        svc_group = [ECS("web1"),
                     ECS("web2"),
                     ECS("web3")]

        with Cluster("DB Cluster"):
            db_master = RDS("userdb")
            db_master - [RDS("userdb readonly")]

        memcached = ElastiCache("memcached")
    
    #create the graph
    dns >> lb >> svc_group
    svc_group >> db_master
    svc_group >> memcached

In [3]:
from diagrams import Cluster, Diagram
from diagrams.aws.compute import EC2
from diagrams.aws.database import RDS
from diagrams.aws.network import ELB
from diagrams.aws.network import Route53
from diagrams.aws.storage import S3
from diagrams.elastic.elasticsearch import Elasticsearch,Logstash

graph_attr = {
    "fontsize": "45"
}

with Diagram("Log Search", direction='TB', show=False, graph_attr=graph_attr):
    dns = Route53("https://searchlogs.example.com")
    lb = ELB("external load balancer")

    with Cluster("SPA"):
        client_group = [EC2("web1"),
                     EC2("web2"),
                     EC2("web3")]

    with Cluster("API"):
        svc_group = [EC2("api1"),
                     EC2("api2"),
                     EC2("api3")]

    with Cluster("DB Cluster"):
        db_master = RDS("master")
        db_master - [RDS("read-replica1")]

    internal_lb = ELB("internal load balancer")

    with Cluster("Ingest Nodes"):
        es_master = Elasticsearch("master")
        ingest_nodes = [es_master,
                        Elasticsearch("ingest-node1"),
                        Elasticsearch("ingest-node2")]

    with Cluster("Query Nodes"):
        query_nodes = [Elasticsearch("query-node1"),
                      Elasticsearch("query-node2")]

    log_bucket = S3('Log Buckets')

    with Cluster("Forwarder"):
        internal_lb - [
            Logstash('forwarder1'),
            Logstash('forwarder2'),
            Logstash('forwarder3')
        ] - log_bucket

    dns >> lb >> client_group
    lb >> svc_group >> internal_lb >> es_master
    internal_lb >> ingest_nodes
    es_master >> query_nodes
    svc_group >> db_master